# Bölüm 1: Temel Prompt Yapısı
- [Ders](__#lesson__)
- [Alıştırmalar](__#exercises__)
- [Örnek Oyun Alanı](__#example-playground__)
## Kurulum
API anahtarınızı yüklemek ve `get_completion` yardımcı fonksiyonunu oluşturmak için aşağıdaki kurulum hücresini çalıştırın.

In [ ]:
%pip install anthropic --quiet

# utils paketinden hints modülünü içe aktar
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Python'un yerleşik düzenli ifade kütüphanesini içe aktar
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt}
        ],
        system=system
    )
    return message.content[0].text

---
## Ders
Anthropic iki API sunar: eski [Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html) ve güncel [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html). Bu eğitim için yalnızca Messages API'yi kullanacağız.
Messages API kullanarak Claude'a yapılan bir çağrı için en azından aşağıdaki parametreler gereklidir:
- `model`: çağırmayı planladığınız modelin [API model adı](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)
- `max_tokens`: durmadan önce üretilecek maksimum token sayısı. Claude'un bu maksimuma ulaşmadan durabileceğini unutmayın. Bu parametre yalnızca üretilecek mutlak maksimum token sayısını belirtir. Ayrıca bu bir *kesin* durdurma olduğundan, Claude'un kelime veya cümle ortasında üretmeyi durdurmasına neden olabilir.
- `messages`: giriş mesajlarından oluşan bir dizi. Modellerimiz, dönüşümlü `user` ve `assistant` konuşma turları üzerinde çalışmak üzere eğitilmiştir. Yeni bir `Message` oluştururken, önceki konuşma turlarını messages parametresiyle belirtirsiniz ve model daha sonra konuşmada bir sonraki `Message`'ı oluşturur.
  - Her giriş mesajı, bir `role` ve `content` içeren bir nesne olmalıdır. Tek bir `user` rolü mesajı belirtebilir veya birden fazla `user` ve `assistant` mesajı ekleyebilirsiniz (bu durumda dönüşümlü olmalıdır). İlk mesaj her zaman `user` rolünü kullanmalıdır.
Ayrıca şu gibi isteğe bağlı parametreler de vardır:
- `system`: sistem promptu - bunun hakkında aşağıda daha fazla bilgi.
- `temperature`: Claude'un yanıtındaki değişkenlik derecesi. Bu dersler ve alıştırmalar için `temperature`'ı 0 olarak ayarladık.
Tüm API parametrelerinin tam listesi için [API dokümantasyonumuzu](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html) ziyaret edin.

### Örnekler
Claude'un doğru biçimlendirilmiş bazı promptlara nasıl yanıt verdiğine bir göz atalım. Aşağıdaki hücrelerin her biri için hücreyi çalıştırın (`shift+enter`) ve Claude'un yanıtı bloğun altında görünecektir.

In [ ]:
# Prompt
PROMPT = "Selam Claude, Nasilsin?"

# Claude cevabini print ediyoruz
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "bana okyanusun ne renk oldugunu soylermisin?"

# Claude cevabini print ediyoruz
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Celine Dion ne zaman dogmusdu?"

# Claude cevabini print ediyoruz
print(get_completion(PROMPT))

Şimdi doğru Messages API biçimlendirmesini içermeyen bazı promptlara bakalım. Bu hatalı biçimlendirilmiş promptlar için Messages API bir hata döndürür.
İlk olarak, `messages` dizisinde `role` ve `content` alanlarının bulunmadığı bir Messages API çağrısı örneğimiz var.

> ⚠️ **Uyarı:** Prompttaki messages parametresinin yanlış biçimlendirilmesi nedeniyle, aşağıdaki hücre bir hata döndürecektir. Bu beklenen bir davranıştır.

In [ ]:
# Claude cevabini aliyoruz
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Merhaba Claude, Nasilsin?"}
        ]
    )

# Claude cevabini print ediyoruz
print(response[0].text)

İşte `user` ve `assistant` rolleri arasında dönüşümlü geçiş yapmayan bir prompt.

> ⚠️ **Warning:** Due to the lack of alternation between `user` and `assistant` roles, Claude will return an error message. This is expected behavior.

In [ ]:
# Claude cevabini aliyoruz
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "Celine Dion ne zaman dogmusdu?"},
          {"role": "user", "content": "ayni zamanda bana onun hakkinda bazi bilgiler ver"}
        ]
    )

# Claude cevabini print ediyoruz
print(response[0].text)

`user` ve `assistant` mesajları **MUTLAKA dönüşümlü olmalıdır** ve mesajlar **MUTLAKA bir `user` turu ile başlamalıdır**. Bir promptta birden fazla `user` & `assistant` çifti bulunabilir (sanki çok turlu bir konuşmayı simüle ediyormuş gibi). Ayrıca, Claude'un kaldığınız yerden devam etmesi için son bir `assistant` mesajına kelimeler koyabilirsiniz (bunun hakkında sonraki bölümlerde daha fazla bilgi).
#### Sistem Promptları
**Sistem promptları** da kullanabilirsiniz. Sistem promptu, "User" turunda Claude'a bir soru veya görev sunmadan önce **bağlam, talimatlar ve yönergeler sağlamanın** bir yoludur.
Yapısal olarak, sistem promptları `user` & `assistant` mesajlarının listesinden ayrı olarak bulunur ve bu nedenle ayrı bir `system` parametresine aittir (not defterinin [Kurulum](__#setup__) bölümündeki `get_completion` yardımcı fonksiyonunun yapısına bakın).
Bu eğitim boyunca, sistem promptu kullanabileceğimiz her yerde, completions fonksiyonunuzda size bir `system` alanı sağladık. Sistem promptu kullanmak istemiyorsanız, `SYSTEM_PROMPT` değişkenini basitçe boş bir dizeye ayarlayın.

#### Sistem Promptu Örneği

In [ ]:
# Sistem promptu
SYSTEM_PROMPT = "Yanıtınız her zaman konuşmayı ileriye taşıyan bir dizi eleştirel düşünme sorusu olmalıdır (sorularınıza cevap vermemelisiniz). Kullanıcının sorusunu gerçekten yanıtlamayın."
# Prompt
PROMPT = "Gokyuzu neden mavi?"

# Claude cevabini print ediyoruz
print(get_completion(PROMPT, SYSTEM_PROMPT))

Neden sistem promptu kullanmalı? **İyi yazılmış bir sistem promptu, Claude'un performansını** çeşitli şekillerde artırabilir, örneğin Claude'un kurallara ve talimatlara uyma yeteneğini artırır. Daha fazla bilgi için, Claude ile [sistem promptlarını nasıl kullanacağınız](__https://docs.anthropic.com/claude/docs/how-to-use-system-prompts__) hakkındaki dokümantasyonumuzu ziyaret edin.
Şimdi bazı alıştırmalara geçeceğiz. Yukarıdaki içeriği değiştirmeden ders promptlarıyla denemeler yapmak isterseniz, [**Örnek Oyun Alanı**](__#example-playground__)'nı ziyaret etmek için ders not defterinin en altına kaydırın.

---

## Alıştırmalar
- [Alıştırma 1.1 - Üçe Kadar Sayma](#exercise-11---counting-to-three)
- [Alıştırma 1.2 - Sistem Promptu](#exercise-12---system-prompt)

### Alıştırma 1.1 - Üçe Kadar Sayma
Uygun `user` / `assistant` biçimlendirmesini kullanarak, Claude'un **üçe kadar saymasını** sağlamak için aşağıdaki `PROMPT`'u düzenleyin. Çıktı ayrıca çözümünüzün doğru olup olmadığını da gösterecektir.

In [ ]:
# Prompt - degistire bileceyin tek yer
PROMPT = "[bu yaziyi degistir]"

# claude cevabini aliyoruz
response = get_completion(PROMPT)

# Alıştırma doğruluğunu değerlendirmek için fonksiyon
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# claude cevabini print ediyoruz ve doğruluğunu değerlendirmek
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("bu alisdirma cozulmusdur:", grade_exercise(response))

❓ İpucu isterseniz, aşağıdaki hücreyi çalıştırın!

In [ ]:
print(hints.exercise_1_1_hint)

### Alıştırma 1.2 - Sistem Promptu
Claude'un 3 yaşında bir çocuk gibi yanıt vermesini sağlamak için `SYSTEM_PROMPT`'u değiştirin.

In [ ]:
# Sistem promptu - degistire bileceyin tek yer
SYSTEM_PROMPT = "[bu yaziyi degistir]"

# Prompt
PROMPT = "gok ne kadar buyuk?"

# claude cevabi
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Alıştırma doğruluğunu değerlendirmek için fonksiyon
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# claude cevabini print ediyoruz ve doğruluğunu değerlendirmek
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("bu alisdirma cozulmusdur:", grade_exercise(response))

❓ İpucu isterseniz, aşağıdaki hücreyi çalıştırın!

In [ ]:
print(hints.exercise_1_2_hint)

### Tebrikler!

eyer simdiyine kadar butun Alistirmalari cozduyseniz, diyer bolume gecmeye hazirsinzi demekdir 

---
## Örnek Oyun Alanı
Bu alan, bu derste gösterilen prompt örnekleriyle özgürce deneme yapmanız ve Claude'un yanıtlarını nasıl etkileyebileceğini görmek için promptları değiştirmeniz için bir alandır.

In [ ]:
# Prompt
PROMPT = "Merhaba Claude, nasılsın?"
# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# claude cevabini al
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Claude'un yanıtını yazdır
print(response[0].text)

In [ ]:
# claude cevabini al
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Claude'un yanıtını yazdır
print(response[0].text)

In [ ]:
# Sistem promptu
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT, SYSTEM_PROMPT))